# Establish working environment and matlab paths

In [ ]:
%%matlab

% Which computer are you on?

if isdir('/Volumes/HD-1/Users/paulmiddlebrooks/')
    projectRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/memory_guided_saccades';
elseif isdir('/Volumes/Macintosh HD/Users/elseyjg/')
    projectRoot = '/Volumes/Macintosh HD/Users/elseyjg/Memory-Guided-Saccade-Project';    
else
    disp('You need to add another condition or the file path is wrong.')
end

dataRoot = fullfile(projectRoot, 'data');
matRoot = fullfile(projectRoot, 'src/matlab');

addpath(genpath(matRoot));
addpath(genpath(fullfile(matRoot,'behavior')));
addpath(genpath(fullfile(matRoot,'mem')));
addpath(genpath(fullfile(matRoot,'neural')));
addpath(genpath(fullfile(matRoot,'plotting')));

% Make this project directory your working directory
cd(matRoot)


# Open a data file

### Open the session list for a subject

In [ ]:
subject = 'joule';

%Open the list of memory guided saccade sessions
fid=  fopen(fullfile(dataRoot,subject, ['mem_sessions_',subject,'.csv']));


nCol = 5;
formatSpec = '%s';
mHeader = textscan(fid, formatSpec, nCol, 'Delimiter', ',');

mData = textscan(fid, '%s %s %d %d %d', 'Delimiter', ',','TreatAsEmpty',{'NA','na'});

sessionList     = mData{1};
hemisphereList  = mData{2};
neuronLogical   = logical(mData{3});
lfpLogical      = mData{4};
eegLogical      = mData{5};

%Extract only sessions with spike data:
sessionList = sessionList(neuronLogical);


### Pick a session and load the data

In [ ]:
sessionInd = 1;
session = sessionList{sessionInd};

[trialData, SessionData] = load_data(subject, session);


# Sort trials based on trial type criteria

In [ ]:


outcome = {'saccToTarget'};
side = {'right'};


trials = mem_trial_selection(trialData, outcome, side);


# Extract spike data (as aligned rasters) for a unit, aligned on event/epoch

In [ ]:
% Set up the variables
unitIndex = 1;
alignEvent = 'targOn';
alignList = trialData.(alignEvent)(trials);
 
    
% Get the rasters (and what index they align to)
[alignedRasters, alignmentIndex] = spike_to_raster(trialData.spikeData(trials, unitIndex), alignList);


# Plot rasters to see each trial's spiking times

In [ ]:
% Set up variables
plotWindow = [-99 : 400];
nTrial = length(trials);


% Open a plot and tell matlab to hold the axes so we can plot multiple things on it
figure(1)
hold all;
colormap(flipud(gray))
hold all;

% Plot the rasters    
imagesc(plotWindow, 1 : nTrial, alignedRasters(:, alignmentIndex + plotWindow))
set(gca,'YDir','normal')

% Plot a vertical line where event occured
plot([0 0], [0 nTrial*.8], '-k', 'lineWidth', 3)

% Change axes limits to whatever you desire
ylim([0 nTrial])
xlim([plotWindow(1) plotWindow(end)])


# Convert rasters to spike density function

In [ ]:
% Define a kernel to convolve rasters- can be 'postsynaptic potential' or 'gaussian', with appropriate parameters:
Kernel.method = 'postsynaptic potential';
Kernel.growth = 1;
Kernel.decay = 20;
% Kernel.method = 'gaussian';
% Kernel.sigma = 10;


sdf = spike_density_function(alignedRasters, Kernel);

sdfMean = nanmean(sdf, 1);


# Plot spike density function over rasters

In [ ]:
% Plot it on top of the rasters
figure(1)
plot(plotWindow, sdfMean(:, alignmentIndex + plotWindow), 'color', 'k', 'lineWidth', 3)

% Alter y-axis to see whole SDF
ylim([0 1.1* max(sdfMean)])


## Wanna see SDFs and rasters for both sides throughout the trial?

In [ ]:
unit = SessionData.spikeUnitArray(unitIndex);

Opt = mem_options;
Opt.printPlot = true;
Opt.unitArray = unit;

Data = mem_session_data(subject, session, Opt);


# Convert spike density function into a heat map plot

In [ ]:
% Normalize the sdf to 1:
sdfNorm = sdfMean ./ max(sdfMean);

% Plot the heatmap on a new figure
figure(2)
colormap('jet')
%imagesc(plotWindow, [0 1], sdfNorm(:, alignmentIndex + plotWindow))
imagesc(sdfNorm(:, alignmentIndex + plotWindow))
caxis([0 1])


# Remove the paths, so you can use your normal codes

In [ ]:
rmpath(matRoot);
rmpath(fullfile(matRoot,'behavior'));
rmpath(fullfile(matRoot,'mem'));
rmpath(fullfile(matRoot,'neural'));
